In [2]:
from main import *

In [2]:
main()

usage: ipykernel_launcher.py [-h] [--data_root DATA_ROOT]
                             [--dataset {voc,cityscapes}]
                             [--num_classes NUM_CLASSES]
                             [--model {deeplabv3_resnet50,deeplabv3plus_resnet50,deeplabv3_resnet101,deeplabv3plus_resnet101,deeplabv3_mobilenet,deeplabv3plus_mobilenet}]
                             [--separable_conv] [--output_stride {8,16}]
                             [--test_only] [--save_val_results]
                             [--total_itrs TOTAL_ITRS] [--lr LR]
                             [--lr_policy {poly,step}] [--step_size STEP_SIZE]
                             [--crop_val] [--batch_size BATCH_SIZE]
                             [--val_batch_size VAL_BATCH_SIZE]
                             [--crop_size CROP_SIZE] [--ckpt CKPT]
                             [--continue_training]
                             [--loss_type {cross_entropy,focal_loss}]
                             [--gpu_id GPU_ID] [--weight

SystemExit: 2

/pkgs/vector_cv_project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 676 kB 5.7 MB/s eta 0:00:01
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655249 sha256=6f68af197715fbaf72cbf901b6a30b3f6da5021250c54e1eeb7d74e28243f965
  Stored in directory: /scratch/ssd002/home/ascherer/.cache/pip/wheels/2d/cd/fb/005445070865d4e45365b2946ee88085a7392370f152cf371c
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5711 sha256=caae6316526c0e7d7bce346229de4a383e641a3d5d2ee982d5a85a349673c5be
  Stored in directory: /scratch/ssd002/home/ascherer/.cache/pip/wheels/55/79/ec/084a3a2e348d72852cc0c13c559c923c13ca54db86e699b681
Successfully built visdom torchfile
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/pkgs/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
from tqdm import tqdm
import network
import utils
import os
import random
import argparse
import numpy as np

from torch.utils import data
from datasets import VOCSegmentation, Cityscapes
from utils import ext_transforms as et
from metrics import StreamSegMetrics

import torch
import torch.nn as nn
from utils.visualizer import Visualizer

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

In [4]:
model_map = {
        'deeplabv3_resnet50': network.deeplabv3_resnet50,
        'deeplabv3plus_resnet50': network.deeplabv3plus_resnet50,
        'deeplabv3_resnet101': network.deeplabv3_resnet101,
        'deeplabv3plus_resnet101': network.deeplabv3plus_resnet101,
        'deeplabv3_mobilenet': network.deeplabv3_mobilenet,
        'deeplabv3plus_mobilenet': network.deeplabv3plus_mobilenet
}

In [6]:
model_map['deeplabv3plus_resnet101'](num_classes=19, output_stride=10)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /h/ascherer/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [9]:
from network.modeling import _segm_resnet, _load_model
#

In [4]:
from network.modeling import _segm_resnet, _load_model
mdl = _load_model('deeplabv3plus', 'resnet101', num_classes=19,
                  output_stride=16, pretrained_backbone=False)
mdl.load_state_dict(torch.load("../../weights/best_deeplabv3plus_resnet101_cityscapes_os16.pth")['model_state'])

<All keys matched successfully>

In [5]:
from network.modeling import _segm_resnet, _load_model
mdl = _load_model('deeplabv3plus', 'mobilenetv2', num_classes=19,
                  output_stride=16, pretrained_backbone=False)
mdl.load_state_dict(torch.load("../../weights/best_deeplabv3plus_mobilenet_cityscapes_os16.pth")['model_state'])

<All keys matched successfully>

In [46]:

mdl._modules['backbone']._modules['conv1'].__dict__

{'_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_parameters': OrderedDict([('weight', Parameter containing:
               tensor([[[[ 1.3288e-02, -3.0716e-03, -1.4754e-02,  ..., -4.9371e-03,
                           4.6551e-03,  1.1708e-02],
                         [-1.1109e-02, -5.6168e-04, -1.8168e-02,  ..., -7.2391e-02,
                          -6.7975e-02, -3.7701e-02],
                         [ 3.9604e-03,  2.7679e-02,  4.6873e-02,  ...,  3.3513e-02,
                           1.1952e-02,  2.4072e-02],
                         ...,
                         [-3.6930e-02, -1.1568e-01, -2.6592e-01,  ..., -3.2814e-01,
                          -1.1980e-01,  2.9107e-02],
                         [ 1.3841e-02,  2.1956e-02,  3.8765e-03,  ..., -1.4526e-01,
                          -1

{'_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_parameters': OrderedDict([('weight', Parameter containing:
               tensor([[[[ 1.3288e-02, -3.0716e-03, -1.4754e-02,  ..., -4.9371e-03,
                           4.6551e-03,  1.1708e-02],
                         [-1.1109e-02, -5.6168e-04, -1.8168e-02,  ..., -7.2391e-02,
                          -6.7975e-02, -3.7701e-02],
                         [ 3.9604e-03,  2.7679e-02,  4.6873e-02,  ...,  3.3513e-02,
                           1.1952e-02,  2.4072e-02],
                         ...,
                         [-3.6930e-02, -1.1568e-01, -2.6592e-01,  ..., -3.2814e-01,
                          -1.1980e-01,  2.9107e-02],
                         [ 1.3841e-02,  2.1956e-02,  3.8765e-03,  ..., -1.4526e-01,
                          -1

In [20]:
wgts = torch.load("../../weights/best_deeplabv3plus_resnet101_cityscapes_os16.pth")['model_state']

In [24]:
.keys()

odict_keys(['backbone.conv1.weight', 'backbone.bn1.weight', 'backbone.bn1.bias', 'backbone.bn1.running_mean', 'backbone.bn1.running_var', 'backbone.bn1.num_batches_tracked', 'backbone.layer1.0.conv1.weight', 'backbone.layer1.0.bn1.weight', 'backbone.layer1.0.bn1.bias', 'backbone.layer1.0.bn1.running_mean', 'backbone.layer1.0.bn1.running_var', 'backbone.layer1.0.bn1.num_batches_tracked', 'backbone.layer1.0.conv2.weight', 'backbone.layer1.0.bn2.weight', 'backbone.layer1.0.bn2.bias', 'backbone.layer1.0.bn2.running_mean', 'backbone.layer1.0.bn2.running_var', 'backbone.layer1.0.bn2.num_batches_tracked', 'backbone.layer1.0.conv3.weight', 'backbone.layer1.0.bn3.weight', 'backbone.layer1.0.bn3.bias', 'backbone.layer1.0.bn3.running_mean', 'backbone.layer1.0.bn3.running_var', 'backbone.layer1.0.bn3.num_batches_tracked', 'backbone.layer1.0.downsample.0.weight', 'backbone.layer1.0.downsample.1.weight', 'backbone.layer1.0.downsample.1.bias', 'backbone.layer1.0.downsample.1.running_mean', 'backbone.

In [ ]:
(classifier): Sequential(
      (0): Conv2d(304, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(256, 19, kernel_size=(1, 1), stride=(1, 1))
    )